<a href="https://colab.research.google.com/github/truboxl/boinc-colab/blob/master/boincappdata_colab_stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BOINC for Google Colab (CPU)

This is a Colab notebook for running BOINC on Google Colab

Written by [truboxl](https://github.com/truboxl)

Licensed under MIT License

See Table of contents at the left sidebar for different sections

In [ ]:
# If you are OK, then "Run all"
# You can try reroll your chance using "Disconnect and delete runtime" but it doesn't always work
# Probability of chip given by Google: (performance not listed in order)
# broadwell:              ++++++++++
# znver2:                 ++
!uptime
!gcc -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -E -- '-march=|-mtune=' | cut -f3 | sort -u
!grep VERSION -nH /etc/os-release
!echo "$(curl -s https://ipinfo.io)"

In [ ]:
# OBSOLETE! See last 2 code blocks at the bottom
# add this code to F12 console to prevent disconnect or use Tampermonkey
# remember to bind F10 key to "Connect to a runtime" in Tools > Keyboard shortcuts
'''
// ==UserScript==
// @name         Colab Auto Reconnect Script
// @namespace    https://github.com/truboxl
// @version      2021-03-07
// @description  Use CARS to stay connected with Google Colab
// @author       truboxl
// @match        *://colab.research.google.com/drive/*
// @grant        none
// ==/UserScript==

(function() {
    'use strict';
    // https://greasyfork.org/en/scripts/412404-colab-保持活跃-make-colab-keep-alive
    // 60010-80490 (1min+) is too short
    function timer() {
        var min = 5*60*1000
        var max = 8*60*1000
        var randomTime = parseInt(Math.random()*(max-min)+min, 10); // 5min+ x 60s x 1000ms
        console.log("Random time (ms) =", randomTime);
        console.log("Random time (min) =", randomTime/1000/60);
        return randomTime;
    }

    // https://www.quora.com/In-JavaScript-how-can-I-simulate-a-keystroke
    var ESCkey = new KeyboardEvent('keydown', {'keyCode':27, 'which':27});
    function FakeESC() {
        console.log("Faking ESC key");
        document.dispatchEvent(ESCkey);
    }

    // https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0
    function FakeClick() {
        console.log("Faking click");
        // no longer works
        //document.querySelector("colab-connect-button").click();
        // assign a key yourself, I choose F10 in keyboard shortcuts for "Connect to a runtime"
        var F10key = new KeyboardEvent('keydown', {'keyCode':121, 'which':121});
        document.dispatchEvent(F10key);
    }
    function FakeEvent() {
        FakeClick();
        console.log("Waiting 5min for next faking");
        console.log("Faking ESC key in 5s");
        setTimeout(FakeESC, 5000);
        setTimeout(FakeEvent, timer());
    }
    // main loop
    setTimeout(FakeEvent, 10*60*1000);
})();
'''

## Connect Google Colab to Google Drive for persistent storage

A Google account is required to allowing storing BOINC data on Google Drive.

ALWAYS MAKE SURE NO ERROR OCCURED HERE!

In [ ]:
%%time
%cd /content/
# https://stackoverflow.com/questions/55918562/changing-the-system-time-in-google-colaboratory
!ln -fs /usr/share/zoneinfo/Asia/Kuala_Lumpur /etc/localtime && date && uptime

# NO LONGER REQUIRED: USE FILES TAB > "MOUNT DRIVE" TO AUTOMOUNT
# allow access to Google Drive for storage, shorten entry to /content/gdrive/
#from google.colab import drive
#drive.mount('drive')
#!uptime && ln -s 'drive/MyDrive' gdrive

# use this without having to sign in every single time after you mount from Files tab
!if [[ -e drive/MyDrive ]]; then exit 1; fi
!if [[ ! -L gdrive ]]; then ln -svT 'drive/MyDrive' gdrive; fi

# you may want to clear your Trash in Google Drive to fix problems
# https://drive.google.com/drive/trash
# https://one.google.com/storage/management

In [ ]:
%%time
# use rsync instead of direct R/W Google Drive to avoid I/O issues
# may take a long time if project has deep folder structure, eg: Rosetta@home
!mkdir -p /content/gdrive/boincappdata/
!rsync -hva --delete-during --inplace /content/gdrive/boincappdata/ /content/boincappdata/ >>/content/init.txt 2>>/content/error.txt

# set up BOINC data directory and permissions
!mkdir -p /content/boincappdata/
%cd /content/boincappdata/
!chmod -R a+rwx projects slots 2>>/content/error.txt
!ls -la projects/*/* slots/*/* | grep 'rw-' && echo "WARN: no executable files found"
#!mount
#!whoami

In [ ]:
# clear error.txt to override
#!echo >/content/content/error.txt

## rsync from Colab to Google Drive periodically

In [ ]:
%%writefile /content/rsync-job.sh
#!/bin/sh
if [ -n "$(cat /content/error.txt)" ]; then echo "ERROR: Something went wrong!" >>/content/error.txt; exit 1; fi
while true; do
    uptime >/content/boincappdata/last-uptime.txt
    rsync -hva --delete-during --inplace /content/boincappdata/ /content/gdrive/boincappdata/ &>>/content/rsync-log.txt
    echo "Done syncing at $(date '+%Y%m%d %r %z')" >>/content/rsync-log.txt
    sleep $((5*60))
done

In [ ]:
# https://amitness.com/2020/06/google-colaboratory-tips/#17-run-background-tasks
print('Previously last uptime:')
!if [ -f /content/boincappdata/last-uptime.txt ]; then cat /content/boincappdata/last-uptime.txt; fi
!chmod a+x /content/rsync-job.sh
!pkill rsync-job.sh
!if [ -z $(pgrep -x rsync-job.sh) ]; then bash -c "nohup chrt -i 0 ionice -c 3 /content/rsync-job.sh &>/dev/null &"; fi

In [ ]:
print('Current last uptime:')
!if [ -f /content/last-uptime.txt ]; then cat /content/last-uptime.txt; fi
!tail /content/rsync-log.txt

# Install dependencies and build tools

In [ ]:
%%time
!echo "$(curl -s https://ipinfo.io)"
#!sed \
#-e "s|^deb http://archive.ubuntu.com/ubuntu/|deb mirror://mirrors.ubuntu.com/US.txt|g" \
#-e "s|^deb http://security.ubuntu.com/ubuntu/|deb mirror://mirrors.ubuntu.com/US.txt|g" \
#-i /etc/apt/sources.list
!sed \
-e "s|^deb http://archive.ubuntu.com/ubuntu/|deb http://azure.archive.ubuntu.com/ubuntu/|g" \
-e "s|^deb http://security.ubuntu.com/ubuntu/|deb http://azure.archive.ubuntu.com/ubuntu/|g" \
-i /etc/apt/sources.list

!apt-get update &>/dev/null
#!apt-get upgrade -y &>/dev/null # can take some time, do we need to update all components? cuda is not updated for some reason
!apt-get install -y git make m4 automake autoconf libtool psmisc libjpeg62 &>/dev/null

##### gcc #####
#!add-apt-repository -y ppa:ubuntu-toolchain-r/test &>/dev/null # gcc 7.5 and clang 6.0 are too old to support skylake+ hardware, latest libstdc++ for clang
#!apt-get install -y gcc-11 g++-11 &>/dev/null # uncomment this if pick gcc

##### clang #####
!curl -sLC- https://apt.llvm.org/llvm.sh -o /content/llvm.sh
!chmod +x /content/llvm.sh
!/content/llvm.sh 21 &>/dev/null
#!curl -s https://apt.llvm.org/llvm-snapshot.gpg.key | apt-key add - # uncomment if llvm.sh does not work
#!add-apt-repository 'deb http://apt.llvm.org/bionic/   llvm-toolchain-jammy-19  main' &>/dev/null # uncomment if llvm.sh does not work
#!apt-get install -y clang-21 &>/dev/null # uncomment if llvm.sh does not work

## Wheel of fortune

In [ ]:
%%time
!lscpu | grep -E 'CPU family:|Model:|Model name:' # sometimes Intel Xeon or AMD EPYC
#!gcc-11 -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -E -- '-march=|-mtune=' | cut -f3 # Xeon has many generations, give me details
#!gcc-11 -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -- '-march=' | cut -f3 >/content/march.txt
!echo | clang-21 -E - -march=native -### # clang is a bit more ugly
!echo | clang-21 -E - -march=native -### 2>&1 | grep /usr/lib | grep -v InstalledDir | sed 's|^.*-target-cpu" "||' | sed 's|".*$||' >/content/march.txt
!cat /content/march.txt

In [ ]:
'''
Tests:
march           |   benchmark (highest ever, usually fluke, fp only, int not used)
broadwell           3170
haswell             3168
znver2              3138
skylake-avx512      3605292K
292K
best config so far:     gcc + -O2 -march=native

Compiler optimization options (listed independently):
Increase benchmark score:           -O3 (stack overflow on znver2 with -flto)
Reduce latency (preferably this):   -O2
Bad:                                -ffast-math
                                    -Os (gcc make tiny executable but very high impact on integer performance, clang is different story)
                                    -Oz (clang only, same as gcc -Os)
Dubious / questionable:             -funroll-loops
Good:                               -march=native (implies -mtune="same_as_march_option" unless CPU newer than database or broken cc)

References:
https://stackoverflow.com/questions/42718572/gcc-mtune-vs-march-vs-mcpu (use -march for x86)
https://community.arm.com/developer/tools-software/tools/b/tools-software-ides-blog/posts/compiler-flags-across-architectures-march-mtune-and-mcpu (use -mcpu for ARM)
https://www.mail-archive.com/boinc_dev@ssl.berkeley.edu/msg08619.html (-flto broke integer benchmark)
https://www.mail-archive.com/boinc_dev@ssl.berkeley.edu/msg08623.html (only floating point score is used for credit calculation)
'''

# Compile BOINC from source

In [ ]:
%%time
# build BOINC from source because I dont really care anymore
%cd /content/
# RIP efficient git protocol, use bulky https now
# https://github.blog/2021-09-01-improving-git-protocol-security-github/#no-more-unauthenticated-git
!git clone --depth 1 https://github.com/boinc/boinc

In [ ]:
%%time
%cd /content/boinc/
import os
commonFLAGS = ' -Oz -march=native' + \
              ' -fstack-protector-strong' + \
              ' -Werror=return-type' + \
              ' -Werror=int-to-pointer-cast' + \
              ' -Werror=pointer-to-int-cast' + \
              ' -Werror=implicit-function-declaration' + \
              ' -ffunction-sections -fdata-sections -fuse-ld=lld -Wl,--gc-sections -Wl,--rosegment'
os.environ['CFLAGS']    = commonFLAGS
os.environ['CXXFLAGS']  = commonFLAGS
'''
!gcc-11 -v 2>&1 | grep 'gcc version'
gcc_version = '11'
os.environ['CC']        = 'gcc-'            + gcc_version
os.environ['CXX']       = 'g++-'            + gcc_version
os.environ['AR']        = 'gcc-ar-'         + gcc_version # important for flto shared
os.environ['RANLIB']    = 'gcc-ranlib-'     + gcc_version # important for flto static
os.environ['NM']        = 'gcc-nm-'         + gcc_version
'''
!clang-21 -v 2>&1 | grep 'clang version'
llvm_version = '21'
os.environ['CC']        = 'clang-'          + llvm_version
os.environ['CXX']       = 'clang++-'        + llvm_version
os.environ['AR']        = 'llvm-ar-'        + llvm_version # important for flto shared
os.environ['RANLIB']    = 'llvm-ranlib-'    + llvm_version # important for flto static
os.environ['NM']        = 'llvm-nm-'        + llvm_version
#!apt-get install libc++{,abi}-20-dev &>/dev/null
#os.environ['CXXFLAGS']  = commonFLAGS       + ' -stdlib=libc++'

!make -s distclean &>/dev/null
!./_autosetup &>/dev/null
!./configure --disable-server --disable-manager &>/dev/null
#!./configure --disable-server --disable-manager \
#             --with-boinc-platform=windows_x86_64 \
#             --with-boinc-alt-platform=x86_64-pc-linux-gnu &>/dev/null

In [ ]:
#%cd /content/boinc/
#!cat config.log

In [ ]:
%%time
%cd /content/boinc/
!make -j$(nproc --all) -s &>/dev/null

# Install / restart BOINC

In [ ]:
%%time
%cd /content/boinc/
!while true; do \
    echo "Checking whether BOINC is running... $(pidof boinc)"; \
    if [ "$(pidof -s boinc)" = "" ]; then \
        break; \
    fi; \
    echo "Killing BOINC..."; \
    kill "$(pidof -s boinc)"; \
    sleep 10; \
done
!make -s install &>/dev/null
!command -v boinc
!du -h /usr/local/bin/boinc*
!strip /usr/local/bin/boinc*
!du -h /usr/local/bin/boinc*
'''
                        -O3         -O2         -Os         -Oz
gcc 11                  1min 49s    1min 45s    1min 40s
/usr/local/bin/boinc    1.4M	    1.1M        848K
/usr/local/bin/boinccmd 412K        352K        292K
/usr/local/bin/boinc    1.2M	    940K        696K
/usr/local/bin/boinccmd 352K        292K        236K
clang 13                1min 21s    1min 9s     1min 10s    1min 11s
/usr/local/bin/boinc    1.1M        1004k       912K        924K
/usr/local/bin/boinccmd 352K        348k        320K        328K
/usr/local/bin/boinc    900K        868k        756K        724K
/usr/local/bin/boinccmd 300K        292k        260K        252K
'''

# Running BOINC

Remove previous logs and run benchmark first so that:

1. credits properly granted
1. compiled BOINC client does not crash

In [ ]:
%cd /content/boincappdata/
#%pycat /content/boincappdata/cc_config.xml
# update device name before launching BOINC rather than after to avoid regen ID
!while true; do \
    echo "Checking whether BOINC is running... $(pidof boinc)"; \
    if [ -z "$(pidof -s boinc)" ]; then \
        break; \
    fi; \
    echo "Killing BOINC..."; \
    kill "$(pidof -s boinc)"; \
    sleep 10; \
done
!echo "Adjusting devicename..."
!cat /content/march.txt
!grep '<device_name>' cc_config.xml
!sed -e "s|<device_name>.*|<device_name>colab $(cat /content/march.txt) $(date +%Y%m%d)</device_name>|" -i cc_config.xml
!grep '<device_name>' cc_config.xml

In [ ]:
%%time
%cd /content/boincappdata/
# if need run benchmark, append boinc with parameter --run_cpu_benchmarks
!if [ -z "$(pidof -s boinc)" ] && [ -z "$(cat /content/error.txt)" ]; then \
    rm -f time_stats_log stdoutdae.txt stderrdae.txt stdoutgpudetect.txt stderrgpudetect.txt; \
    echo 'Starting BOINC...'; \
    boinc --daemon --dir /content/boincappdata/ --gui_rpc_port 31416; \
    sleep 30; \
fi
!if [ -z "$(pidof -s boinc)" ]; then echo 'BOINC is not starting, check /content/error.txt, you might want to delete runtime and run all again'; exit 1; fi

# Managing BOINC

## Output

Running the first 2 should give you the overall view of VM usage by BOINC

In [ ]:
%%time
!uptime
!pstree -al # check currently running processes
#!cat /proc/meminfo # check meminfo

In [ ]:
%%time
%cd /content/boincappdata/
!uptime
#!cat stdoutdae.txt # show full log
#!cat stderrdae.txt
#!tail stdoutdae.txt # show last few lines
#!tail -f stdoutdae.txt # follow log
#!tail -n30 stdoutdae.txt # last 30 lines of log
!tail -n15 stdoutdae.txt # last 15 lines of log

## Control panel (Attach Project)

In [ ]:
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --lookup_account URL email passwd
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project_attach URL auth
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --acct_mgr attach URL name passwd
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --acct_mgr sync
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --acct_mgr info
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --acct_mgr detach

## Control panel (boinccmd)

In [ ]:
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project https://einstein.phys.uwm.edu/ allowmorework
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project https://einstein.phys.uwm.edu/ nomorework
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project http://www.worldcommunitygrid.org/ allowmorework
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project http://www.worldcommunitygrid.org/ nomorework
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --task http://www.worldcommunitygrid.org/ MCM1_0244716_3424_1 suspend && sleep 10
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --task http://www.worldcommunitygrid.org/ MCM1_0244716_3424_1 resume

In [ ]:
%%time
%cd /content/boincappdata/
!uptime && du -sh
#!boinc --help
#!boinccmd --help
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --network_available
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --project URL allowmorework # reset | detach | update | suspend | resume | nomorework | allowmorework | detach_when_done | dont_detach_when_done
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --task URL task_name abort # suspend | resume | abort
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --get_state #| grep 'request more work'
!boinccmd --passwd $(cat gui_rpc_auth.cfg) --get_tasks | \
    grep -E '   name|project URL|report deadline|active_task_state|CPU time|fraction done' | \
    grep -v 'CPU time remaining' | \
    sed -e 's|^   name:|name:|'

In [ ]:
%%time
!uptime && du -sh
#!find /content/boincappdata/slots -name stdout.txt | sort | xargs -i bash -c 'echo "===== {} =====" && tail "{}"'
!find /content/boincappdata/slots -name stderr.txt | sort | xargs -i bash -c 'echo "===== {} =====" && tail "{}"'

## Control panel (cc_config.xml)

In [ ]:
#%cd /content/boincappdata/
#%pycat /content/boincappdata/cc_config.xml
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --read_cc_config

In [ ]:
'''
%%writefile /content/boincappdata/cc_config.xml
<cc_config>
<options>
<device_name>colab</device_name>
<report_results_immediately>0</report_results_immediately>
<no_alt_platform>1</no_alt_platform>
</options>
</cc_config>
'''

## Control panel (global_prefs_override.xml)

In [ ]:
#%cd /content/boincappdata/
#%pycat /content/boincappdata/global_prefs_override.xml
#!boinccmd --passwd $(cat gui_rpc_auth.cfg) --read_global_prefs_override

In [ ]:
'''
%%writefile /content/boincappdata/global_prefs_override.xml
<global_preferences>
   <run_on_batteries>1</run_on_batteries>
   <run_if_user_active>1</run_if_user_active>
   <run_gpu_if_user_active>1</run_gpu_if_user_active>
   <suspend_cpu_usage>0.000000</suspend_cpu_usage>
   <start_hour>0.000000</start_hour>
   <end_hour>0.000000</end_hour>
   <net_start_hour>0.000000</net_start_hour>
   <net_end_hour>0.000000</net_end_hour>
   <leave_apps_in_memory>1</leave_apps_in_memory>
   <confirm_before_connecting>0</confirm_before_connecting>
   <hangup_if_dialed>0</hang5 up_if_dialed>
   <dont_verify_images>0</dont_verify_images>
   <work_buf_min_days>0.000000</work_buf_mCopy of in_days>
   <work_buf_additional_days>0.000000</work_buf_additional_days>
   <max_ncpus_pct>100.000000</max_ncpus_pct>
   <cpu_scheduling_period_minutes>60.000000</cpu_scheduling_period_minutes>
   <disk_interval>60.000000</disk_interval>
   <disk_max_used_gb>0.000000</disk_max_used_gb>
   <disk_max_used_pct>100.000000</disk_max_used_pct>
   <disk_min_free_gb>0.000000</disk_min_free_gb>
   <vm_max_used_pct>1.000000</vm_max_used_pct>
   <ram_max_used_busy_pct>75.000000</ram_max_used_busy_pct>
   <ram_max_used_idle_pct>90.000000</ram_max_used_idle_pct>
   <max_bytes_sec_up>0.000000</max_bytes_sec_up>
   <max_bytes_sec_down>0.000000</max_bytes_sec_down>
   <cpu_usage_limit>100.000000</cpu_usage_limit>
   <daily_xfer_limit_mb>0.000000</daily_xfer_limit_mb>
   <daily_xfer_period_days>0</daily_xfer_period_days>
</global_preferences>
'''

In [ ]:
#%cd /content/boincappdata/
#%pycat /content/boincappdata/client_state.xml

In [ ]:
#%%writefile /content/boincappdata/client_state.xml

# Let Colab run indefinitely

In [ ]:
# WORK IN PROGRESS (RUN AS SEPARATE ACCOUNT)
#!deluser --remove-all-files boinc
#!delgroup boinc
#!addgroup boinc
#!adduser --no-create-home --disabled-password --gecos "boinc" boinc
#!chown -hR boinc:boinc /content/boincappdata/
#!chmod -R a+rw /content/boincappdata/
#!chmod -R a-x /content/boincappdata/
#!chmod -R o+rwx /content/boincappdata/projects/ /content/boincappdata/slots/
#!runuser -u boinc command

In [ ]:
#!rsync -hva --delete --delete-during --inplace /content/boincappdata/ /content/gdrive/boincappdata/ #&>>/content/rsync-log.txt

In [ ]:
%%time
%cd /content/boincappdata/
!uptime
!gcc -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -E -- '-march=|-mtune=' | cut -f3 | sort -u
!boinccmd --passwd $(cat gui_rpc_auth.cfg) --get_tasks | \
    grep -E '   name|project URL|report deadline|active_task_state|CPU time|fraction done' | \
    grep -v 'CPU time remaining' | \
    sed -e 's|^   name:|name:|'
!tail -f stdoutdae.txt # follow log